In [4]:
import tensorflow as tf
import numpy as np
import sys
'''
기본 원칙!
1. 큐가 있고
2. 큐에 값을 채워넣을 operation이 있고
3. 큐가 빌 때 operation을 반복 실행시켜줄 queue runner가 있어야 한다.
5. queue runner는 쓰레드를 사용하는데
4. 그 쓰레드는 coordinator에 의해 관리된다. 
'''
QUEUE_LEN = 20
#1. 큐가 있고
q = tf.FIFOQueue(QUEUE_LEN,"float")
#2. 큐에 값을 채워넣을 operation이 있고
enq_ops1 = q.enqueue_many(([1.,1.],))
enq_ops2 = q.enqueue_many(([2.,2.],))
enq_ops3 = q.enqueue_many(([3.,3.],))
#3. 큐가 빌 때 operation을 반복 실행시켜줄 queue runner가 있어야 한다.
qr = tf.train.QueueRunner(q,[enq_ops1,enq_ops2,enq_ops3])

sess = tf.Session()
#4. 쓰레드는 coordinator에 의해 관리된다.
coord = tf.train.Coordinator()
#5. queue runner는 쓰레드를 사용한다
threads = qr.create_threads(sess,coord=coord,start=True)

for step in range(20):
    print(sess.run(q.dequeue()))
#모든 쓰레드들을 정지 시킨다.
coord.request.stop()
#다음 코드를 진행하기전에, Queue Runner의 모든 쓰레드들이 정지될때 까지 기다린다.
coord.join(threads)

        

In [ ]:
#실용적인 코드

#1. 큐 생성(#2 operation이 내포된)
filename_queue = tf.train.string_input_producer(["1","2","3"],shuffle=False)

with tf.Session() as sess:
    #4. 쓰레드는 coordinator에 의해 관리된다.
    coord = tf.train.Coordinator()
    #5. queue runner는 쓰레드를 사용한다. #3 qr과 쓰레드 동시 생성
    threads = tf.train.start_queue_runners(coord=coord,sess=esss)
    
    for step in range(20):
        print(sess.run(filename_queue.dequeue()))
    
    #모든 쓰레드들을 정지 시킨다.
    coord.request.stop()
    #다음 코드를 진행하기전에, Queue Runner의 모든 쓰레드들이 정지될때 까지 기다린다.
    coord.join(threads)
        
    

In [ ]:
#reader와 decoder 도입한 코드

filename_queue = tf.train.string_input_producer(["1"],["2"],["3"], shuffle=False,
                                               name='filename_queue')
#dequeue 대신 파일에서 데이터를 읽어오는 Reader
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

#value를 parsing하는 decoder
record_defaults = [ ["null"],[1],[1900],["null"],["null"]]

ID, num, year, rtype , rtime = tf.decode_csv(value, record_defaults=record_defaults,
                                             field_delim=',')
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(100):
        print(sess.run([ID, num, year, rtype , rtime]))
    coord.request_stop()
    coord.join(threads)

In [ ]:
#image주소,label 꼴로 저장된 csv 파일 읽고 파싱하기

csv_file = tf.train.string_input_producer(["1"],["2"],["3"], shuffle=False,
                                               name='filename_queue')
textReader = tf.TextLineReader()
_,line = textReader.read(csv_file)
imagefile,label = tf.decode_csv(line,record_default=['',''])
image = tf.image.decode_jpeg(tf.read_file(imagefile),channels=3)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for i in range(100):
        image_value, label_value, imagefile_value = sess.run([image,label,imagefile])
        plt.imshow(image_value)
        plt.show()
        print(label_value,imagefile_value)
    coord.request_stop()
    coord,join(threads)

In [ ]:
#배치 처리를 도입한 코드
#데이타 포맷: 2014,xv,121
#결론은 데이터 하나를 읽는 법을 먼저 정의한 다음, batch를 통해 여러 개를 읽는다는 것!

def read_data(file_name):
    try:
        csv_file = tf.train.string_input_producer([file_name],name='filename_queue')
        textReader = tf.TextLineReader()
        _,line = textReader.read(csv_file)
        year,flight,time = tf.decode_csv(line,record_default=[[1900],[''],[0]])
    except:
        print("error")
        exit()
    return year,flight,time

#입력 텐서의 shape이  [x,y,z] 일 경우 
#tf.train.batch를 통한 출력은 [batch_size,x,y,z] 가 된다
def read_data_batch(file_name,batch_size=10):
    year, flight, time = read_data(file_name)
    batch_year,batch_flight,batch_time = tf.train.batch([year,flight,time],
                                                       batch_size=batch_size)
    return batch_year,batch_flight,batch_time 

def main():
    #coordinate 위에 코드가 있어햐 한다
    #데이타를 집어넣기 전에 미리 그래프가 만들어져 있어야 함!
    batch_year, batch_flight,batch_time = read_data_batch(TRAINING_FILE)
    year = tf.placeholder(tf.int32,[None,])
    flight = tf.placeholder(tf.string,[None,])
    time = tf.placeholder(tf.int32,[None,])
    
    tt = time*10
    
    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        
        #run을 돌려서 데이터를 받음
        y_,f_,t_ = sess.run([batch_year, batch_flight,batch_time])
        #실행 run
        print(sess.run(tt,feed_dict={time:t_}))
    coord.request_stop()
    coord,join(threads)
        
    

In [ ]:
#요렇게도 할 수 있다.

def get_input_queue(csv_file_name,num_epochs = None):
    train_images = []
    train_labels = []
    for line in open(csv_file_name,'r'):
        cols = re.split(',|\n',line)
        train_images.append(cols[0])
        # 3rd column is label and needs to be converted to int type
        train_labels.append(int(cols[2]) )
    input_queue = tf.train.slice_input_producer([train_images,train_labels],
                                               num_epochs = num_epochs,shuffle = True)

    return input_queue
'''
get_input_queue 함수에, csv_file_name을 인자로 주면, 
이 파일을 한줄 단위로 읽어서, 첫번째는 파일명, 세번째는 라벨로 읽은 후에, 
각각 train_images와  train_lables에 각각 string과 int 형으로 저장한다
그 다음이 배열을 가지고 tf.train.slice_input_producer를 사용하면 
배열에서 데이타를 읽어 드리는 input queue 를 생성하는데, 
이때 인자로 shuffle = True로 주면 데이타를 리턴 할때 순차적으로 
리턴하지 않고 셔플된 형태로 랜덤하게 리턴한다. 
'''
def read_data(input_queue):
    image_file = input_queue[0]
    label = input_queue[1]
    image =  tf.image.decode_jpeg(tf.read_file(image_file),channels=FLAGS.image_color)
    return image,label,image_file

'''
다음으로, 이 큐를 이용하여 이미지 파일명과, 라벨을 읽어서 
이미지 파일 데이타(텐서)와 라벨로 읽는 코드를 read_data라는 함수로 구현하였다. 
입력값은 input_queue인데, input queue에서 데이타를 읽으면 첫번째는 이미지 파일명, 
두번째는 라벨이 되는데, 첫번째 파일명을 tf.image.decode_jpeg함수를 이용하여 
텐서로 읽은후, 읽은 이미지 데이타와 라벨을 리턴하였다.
'''

def read_data_batch(csv_file_name,batch_size=FLAGS.batch_size):
    input_queue = get_input_queue(csv_file_name)
    image,label,file_name= read_data(input_queue)
    image = tf.reshape(image,[FLAGS.image_size,FLAGS.image_size,FLAGS.image_color])
    batch_image,batch_label,batch_file = tf.train.batch([image,label,file_name],batch_size=batch_size)

                                                       #,enqueue_many=True)
    batch_file = tf.reshape(batch_file,[batch_size,1])
    batch_label_on_hot=tf.one_hot(tf.to_int64(batch_label),
        FLAGS.num_classes, on_value=1.0, off_value=0.0)
    return batch_image,batch_label_on_hot,batch_file

'''
마지막으로, 배치로 데이타를 읽는 함수 부분에서 앞에 정의한 get_input_queue와 
read_data 함수를 이용하여 데이타를 shuffle 된 상태로 읽은 후에, 
tf.train.batch를 이용하여 일정한 개수 (배치) 형태로 리턴하도록 하였다.
'''


#출처: http://bcho.tistory.com/1174?category=555440 [조대협의 블로그]
